# Toranto clustering project

 build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe
 

First we import all libraries that we will use in our scraping and cleaning operation

In [233]:
import urllib.request, urllib.parse, urllib.error
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup# Scrap Wikipedia
import pandas as pd

In order to obtain the data that is in the Wikipedia table of postal codes, we propose to use **BeautifulSoup** library:

1. first identify the URL to scrap in cururl variable
2. open a request to this url and save the response in data variable
3. extract table tag of class="wikitable sortable" and then explore each row and column of this table
4. store obtained data in a Dataframe named df_codes

In [234]:


cururl = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

print('=== URLS Yet To Retrieve:', len(cururl))
 
 
print('RETRIEVING', cururl)
data =  urllib.request.urlopen(cururl).read() 
 
soup = BeautifulSoup(data)
tables = soup.find("table",{"class":"wikitable sortable"}) # wikitable sortable jquery-tablesorter
# print 'Tags'
 

=== URLS Yet To Retrieve: 63
RETRIEVING https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M


To clean and prepare our data frame, we use a dictionary structure, which allows us to search easly each postal code and find the associated borough and neighborhood, we apply thereafter some conditionnal test as follow :

1. check that each borough is diffrent to 'Not assigned'
2. if neighborhood is equal to  'Not assigned' replace the neighborhood value by the borough value
3. chech if the code is already added in the dictionary, if yes we concatenate the neighborhood with the existing one using comma
4. remove special charachter using strip()
 

In [235]:
tags_tr= tables.findAll('tr') 

postalcodes={}
for tr in tags_tr:
    #print(tr)
    tds = tr.findAll('td')
    
    if tds is not None and len(tds) > 0:
        for code, borough, neigh in zip(tds[0],tds[1],tds[2]) :
            if ( borough!= 'Not assigned'):
                try :
                    borough = borough.text
                    
                except:
                    None
                try :
                    neigh = neigh.get('title')                    
                except:
                    None
                if ( neigh.strip() == r'Not assigned'):
                        neigh= borough

                        
                try :                     
                    if (code in postalcodes.keys() ):
                        neigh =postalcodes.get(code)[2] + ',' + neigh  

                except:
                    None
                
                postalcodes[code]=[code.strip(),borough.strip() , neigh.strip()]

In [236]:
df_codes = pd.DataFrame(postalcodes.values(),columns=['postalcode','borough','neighborhood'])
df_codes

,postalcode,borough,neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront (Toronto),Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Toronto,Malvern, Toronto"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


In the last we build a dataframe using the scraped data and give proper names to the 3 columns, then we found that we have 103 rows.

In [237]:
df_codes.shape

(103, 3)